<a href="https://colab.research.google.com/github/JMML2021/0401_Registre-de-logs/blob/main/0401_Registre-de-logs%20Rev%20C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [283]:
# importing libraries
from urllib.request import urlretrieve
import pandas as pd
import os
import re
from datetime import datetime
import pytz
#import geocoder
import folium
import time
import urllib.request
import json
#import geopandas
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()


https://stackoverflow.com/questions/65324533/geopandas-in-google-colab

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 12.8 MB/s 
     |████████████████████████████████| 16.7 MB 18.5 MB/s 
     |████████████████████████████████| 6.3 MB 44.0 MB/s 


In [284]:
# Activo Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




```
# Tiene formato de código
```

# Nivell 1
L'analista ha d'assegurar-se que els registres consisteixen en una gamma completa de missatges i s'interpreten segons el context. Els elements de registre han d'estandaritzar-se, utilitzant els mateixos termes o terminologia, per evitar confusions i proporcionar cohesió.

Com Científic de Dades se t'ha proporcionat accés als registres-Logs on queda registrada l'activitat de totes les visites a realitzades a la pàgina web de l'agència de viatges "akumenius.com".


In [285]:
# Abro archivo

df = list(open('/content/drive/MyDrive/Web_access_log-akumenius.com.txt'))
df[200:210]

['www.akumenius.com 180.76.5.31 - - [23/Feb/2014:03:15:03 +0100] "GET /destinos-baratos/destinos-caracteristicas/hoteles-baratos-en-Zurich_SUIZA-con-Alquiler-de-silla-de-ruedas HTTP/1.1" 200 8360 "-" "Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:03 +0100] "GET /hoteles-baratos/ofertas-hotel-Capitolio-Apartamentos-Turisticos-en-Merida-207271b-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /destinos-baratos/destinosEstrelles/hoteles-en-Gary_ESTADOS%20UNIDOS-con-4-estrellas HTTP/1.1" 200 8812 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /hoteles-baratos/ofertas-hotel-Guitart-Termes-La-Collada-en-La%20Molina-6833b-destinos.

# Creo un fichero de IPs chopeadas del original

In [286]:
print('NUmero de registros:')
len(df)

NUmero de registros:


261873

# Exercici 1
Estandaritza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [287]:
#2.- Separo la parte central.

def custom_split(str_to_split):
    separatorsCentro = "[", "]", "\s", ",",' ','+', ')', '(', '"'
    # create regular expression dynamically
    regular_exp = '|'.join(map(re.escape, separatorsCentro))

    return re.split(regular_exp, str_to_split)

In [288]:
def separoCorchete ( textoCortar):
  separatorsCentro = "]"
  regular_exp = '|'.join(map(re.escape, separatorsCentro))

  return re.split(regular_exp, textoCortar)



In [289]:
def arregloFecha(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
    timezone will be obtained using the `pytz` library.
    '''
    dt = datetime.strptime(x, '%d/%b/%Y:%H:%M:%S')

    #dt = datetime.strptime(x[0:-9], '%d/%b/%Y:%H:%M:%S')
    #print('Fecha ', dt)
    #dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
    #return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))
    return dt

In [305]:
#2.- Separo la parte central.

def custom_split_Dcha(str_to_split):
    #separatorsCentro = "[", "]", "\s", ",",' ','+', ')', '(', '"'
    separatorsCentro = "[", "]", '"'



    # create regular expression dynamically
    regular_exp = '|'.join(map(re.escape, separatorsCentro))

    return re.split(regular_exp, str_to_split)

In [311]:
df1 = df.copy()
data = {'virtual_host': [],    'IPs': [],    'Fecha': [],    'request': [],  'status': [], 'size': [], 'referer': [], 'user_agent': []}

df_salida = pd.DataFrame(data)
print('lineas total ------> ', len(df1))

for s, textoLinea  in enumerate(df1):
  #if len(textoLinea) == 23:
    #print ('---', s, textoLinea) 
  problematico= 0
  if s<5000:
    textin = separoCorchete(textoLinea)
    if len(textin) > 2:
      textin[1] = textin[1] +textin[2]


    print(s, len(textin), textin[1])
    texto = custom_split(textin[0])

    textoDcha= custom_split_Dcha(textin[1])
    #for p, q in enumerate(textoDcha):
     #print(p,q)

    
    nueva_fila = {'request': textoDcha[1],
            'status': textoDcha[2],
            'size': textoDcha[3], 
            'referer': textoDcha[5]}
            

Se han truncado las últimas 5000 líneas del flujo de salida.
2500 2  "GET /destinos-caracteristicas/hoteles-baratos-en-La-Gomera_ESPA%C3%91A-con-Cambio-de-divisas HTTP/1.1" 200 7236 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-

2501 2  "GET /destinosCaracteristicas/hoteles-baratos-en-Ho-Chi-Minh-City-(Saigon)-and-South_VIETNAM-con-WIFI HTTP/1.1" 200 7218 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-

2502 2  "GET /hoteles-baratos/ofertas-puente-de-mayo.html HTTP/1.1" 200 6125 "http://www.akumenius.com/" "Mozilla/5.0 (compatible; spbot/4.0.7; +http://OpenLinkProfiler.org/bot )" VLOG=-

2503 2  "GET /destinosCaracteristicas/hoteles-baratos-en-Sligo_IRLANDA-con-Cafeter%C3%ADa HTTP/1.1" 200 7929 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-

2504 2  "GET /hoteles-baratos/ofertas-hotel-Hunters-Hall-en-TETBURY-136214t-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (comp


Once the lines are printed, we see they are in the lines shows Common Log Format (CLF) :

check https://httpd.apache.org/docs/2.2/logs.html what CLF means  

He dividido cada linea es 2 partes para tener mejor control y reducir el numero de accesos que hace con cada "spliter" elemento.
El lado Izq va hasta que encuentra el ']' y la derecha es el otro lado y lo separare por ".


In [316]:
import re
 
chopear = input('Quieres abrir el fichero de nuevo y?')
if chopear == 'y' or chopper =='Y':
  texto=[]

    
  df1 = df.copy()
  data = {'virtual_host': [],    'IPs': [],    'Fecha': [],    'request': [],  'status': [], 'size': [], 'referer': [], 'user_agent': []}

  df_salida = pd.DataFrame(data)
  print('lineas total ------> ', len(df1))

  #for s, textoLinea  in enumerate(df1):
  #  print('Hola', s, textoLinea)

  for s, textoLinea  in enumerate(df1):
    #if len(textoLinea) == 23:
      #print ('---', s, textoLinea) 
    problematico= 0
    if s<10000000:
      textin = separoCorchete(textoLinea)

      #print(s, len(textin), textin[0])
      texto = custom_split(textin[0])
      if len(textin) > 2:
        textin[1] = textin[1] +textin[2]

      nueva_fila = {'virtual_host': texto[0],
            'IPs': texto[1],
            'Fecha': arregloFecha(texto[5])}  # Lo tranformo en fecha

      textoDcha= custom_split_Dcha(textin[1])
    #for p, q in enumerate(textoDcha):
     #print(p,q)

    
    nueva_fila = {'virtual_host': texto[0],
            'IPs': texto[1],
            'Fecha': arregloFecha(texto[5]),
            'request': textoDcha[1],
            'status': textoDcha[2],
            'size': textoDcha[3], 
            'referer': textoDcha[5]}

      #for p,q in enumerate(texto):
        #print(p, q)

    if s%10000 ==0 :
            print(s, 'Z--->', nueva_fila)
            print()
    #df_salida=pd.DataFrame.from_dict(nueva_fila, orient='index') 
    df_salida= df_salida.append(nueva_fila, ignore_index=True) 

  # Guardo la informacion en drive   
  path = ('/content/drive/MyDrive/01_COLAB/' +'direcionesIP_otro.csv')
  df_salida.to_csv(path)
  #
  #df_salida.type
  print('Forma ', df_salida.shape)

  df_salida.head(-20)


Quieres abrir el fichero de nuevo y?y
lineas total ------>  261873
0 Z---> {'virtual_host': 'localhost', 'IPs': '127.0.0.1', 'Fecha': datetime.datetime(2014, 2, 23, 3, 10, 31), 'request': 'OPTIONS * HTTP/1.0', 'status': ' 200 - ', 'size': '-', 'referer': 'Apache (internal dummy connection)'}

10000 Z---> {'virtual_host': 'www.akumenius.com', 'IPs': '144.76.95.232', 'Fecha': datetime.datetime(2014, 2, 23, 10, 24, 39), 'request': 'GET /destinos-baratos/hoteles-baratos/ofertas-hotel-NH-Palacio-de-Tepa-en-Puerta%20del%20Sol-Plaza%20Mayor-137745b-destinos.html HTTP/1.0', 'status': ' 404 3100 ', 'size': '-', 'referer': 'Mozilla/5.0 (compatible; MJ12bot/v1.4.4; http://www.majestic12.co.uk/bot.php?+)'}

20000 Z---> {'virtual_host': 'www.akumenius.com', 'IPs': '180.76.5.171', 'Fecha': datetime.datetime(2014, 2, 23, 15, 14, 11), 'request': 'GET /destinos-baratos/destinos-caracteristicas/hoteles-baratos-en-Ceuta_ESPA%C3%91A-con-Habitaciones-climatizadas HTTP/1.1', 'status': ' 200 7214 ', 'size': 

In [317]:
print(type(df_salida))
print(df_salida.dtypes)
print('Tmaño: ', df_salida.shape)

<class 'pandas.core.frame.DataFrame'>
virtual_host            object
IPs                     object
Fecha           datetime64[ns]
request                 object
status                  object
size                    object
referer                 object
user_agent             float64
dtype: object
Tmaño:  (261873, 8)


# Abrir fichero de IPs creado

In [319]:
path = ('/content/drive/MyDrive/01_COLAB/' +'direcionesIP.csv')
df=  pd.read_csv(path)
print('Tamaño antes de tomar una muestra: ', df.shape)

#df = df.sample(frac = 0.5)
#df = df[1294:1305]
print(type(df))
print('Tamaño: ', df.shape)

Tamaño antes de tomar una muestra:  (261873, 9)
<class 'pandas.core.frame.DataFrame'>
Tamaño:  (261873, 9)


# Exercici 2
Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

Lo primero que voy a hacer es minimizar la base de datos.
La manera que se me ha ocurrido es poner un contador por Ip y asi si hay IP repetidas solo la buscará una vez y tendre la cantidad en el df

In [320]:
data_ip =df["IPs"].value_counts().rename_axis('ip').reset_index(name="visits")
data_ip.head()

,ip,visits
0,66.249.76.216,46382
1,80.28.221.123,14725
2,127.0.0.1,13892
3,217.125.71.222,5201
4,66.249.75.148,3558


In [321]:
!pip install geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [322]:
import geocoder

ip = geocoder.ip("157.55.32.183")
print(ip.city)
print(ip)
print(ip.latlng)

Cleveland
<[OK] Ipinfo - Geocode [Cleveland, Ohio, US]>
[41.4995, -81.6954]


# Exercici 3
Geolocalitza les IP's.

In [323]:

location = geocoder.ip('66.249.76.216')

In [324]:
posicion = []

for index, value in data_ip["ip"].items():
    with urllib.request.urlopen("https://geolocation-db.com/jsonp/"+value,timeout=500) as url:
                data = url.read().decode()
                data = data.split("(")[1].strip(")")
                data = json.loads(data)
                posicion.append(data)


KeyboardInterrupt: ignored

In [ ]:
posicion = pd.DataFrame(posicion)
 
type(posicion)


In [ ]:


!pip install geopandas
import geopandas

In [ ]:
posicion['cantidad'] = data_ip['visits']
posicion = posicion[posicion['state']!= 'Not found']
posicion

In [ ]:
  # Guardo el archivo con la geoposiciones

  pathPosicion = ('/content/drive/MyDrive/01_COLAB/' +'GEOdirecionesIP.csv')

In [ ]:
  sobreEscribir = input(Quiere sobre escribir el fichero? y/Y)
  if sobreEscribir=='Y' or sobreEscribir=='y':
    posicion.to_csv(path)

Si ya hemos geolocacliadas todas las Ips, entonces solo tenemos que descar el fichero y no hace falta repetir todos los pasos anteriores

In [ ]:
# Abro archivo

posicion = pd.read_csv(pathPosicion)
posicion[200:210]

In [ ]:
posicion = pd.DataFrame(posicion)
 
type(posicion)

In [ ]:
posicion.head(-1)

In [ ]:
# Elimino todos los registros que no tengan informacion
# por ejemplo los 'not found'

posicion['cantidad'] = data_ip['visits']
posicion = posicion[posicion['latitude']!= 'Not found']
posicion.shape

In [ ]:
posicion['latitude'].astype(float, errors = 'raise')
posicion.info()

In [ ]:
gdf = geopandas.GeoDataFrame(
    posicion, geometry=geopandas.points_from_xy(posicion.longitude,posicion.latitude))

In [ ]:
gdf.head(-1)

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.set_aspect('equal')

world.plot(ax=ax, color='white', edgecolor='black')
ax.set_axis_off()

gdf.plot(ax=ax, marker='o', color='red', markersize=5)
plt.show()

In [ ]:
! pip install folium

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map(location=[40.965, -5.664], zoom_start=3)
mc = MarkerCluster()

In [ ]:
#! pip install geocoder

In [ ]:
for i in range(0,len(posicion)):
    mc.add_child(folium.Marker(
        location=[posicion.iloc[i]["latitude"],posicion.iloc[i]["longitude"]],
        popup=str(posicion.iloc[i]["cantidad"]),))

In [ ]:
m.add_child(mc)

In [ ]:
https://datasciencesphere.com/project/track-location-ip-address-python-geocoder/

# Exercici 3
Mostra'm la teva creativitat, Sorprèn-me fes un pas més enllà amb l'anàlisi anterior.

https://datasciencesphere.com/project/track-location-ip-address-python-geocoder/

In [ ]:
Spain = posicion[(posicion['country_name'] == 'Spain') | (posicion['country_name'] == 'Portugal')]

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map(location=[40.965, -5.664], zoom_start=7)
mc = MarkerCluster()

In [ ]:
Spain

In [ ]:
posiciones=[]
#import webbrowser

from folium import plugins
#feature_group = folium.FeatureGroup(name='Cantidad de accesos IP') 

mapa = folium.Map(location=[40.4167, -3.70325], zoom_start=4.9) #, width= 800, height =700)

for i in range(0,len(Spain)):
  posiciones.append([Spain.iloc[i]["latitude"],Spain.iloc[i]["longitude"]])

#print('\n Cantidad de accesos de IPs')
mapa.add_child(plugins.HeatMap(posiciones[:]))#.add_to(feature_group)
#mapa.add_child(folium.map.LayerControl())

'''
# Inicio Titulo
html="<h1 class='text-centre'>Cantidad de Ips</h1>";
titulo= folium.Element(html);
mapa.get_root().html.add_child(titulo);

# Guardar Mapa
archivoHTML ='fotoCalorCantidadAccesosIPs'
mapa.save(archivoHTML)

#abrir navegador
chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s'
webbrowser.get(chrome_path).open(archivoHTML)

#webbrowser.open(archivoHTML, new=0, autoraise=True)
'''